In [5]:
#%pip install --upgrade langchain langsmith langgraph langchain_openai openai python-dotenv javascript
# run this in terminal: python3 -m javascript --install @near-lake/primitives
%reload_ext autoreload
%autoreload 2

In [8]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv('.env')

# Set model variables
OPENAI_BASE_URL = "https://api.openai.com/v1"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_ORGANIZATION = os.getenv("OPENAI_ORGANIZATION")


os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "Indexer Agent Local"


In [80]:
from langchain_core.messages import HumanMessage
from tools.JavaScriptRunner import tool_js_on_block



inputs = {
    "messages": [
        HumanMessage(
            content="Build javascript code that returns all function calls with their arguments, block number, date and receiptId",
        )
    ]
}

from graph.indexer_agent_graph import IndexerAgentGraphBuilder
from agents.IndexerAgent import indexer_agent_model, tool_executor

# Define the tools that the agent will use
tools = [tool_js_on_block]
model = indexer_agent_model(tools)

# print(model.invoke(input=inputs['messages']))

builder = IndexerAgentGraphBuilder(model, tool_executor(tools))
app = builder.graph()

for output in app.with_config({"run_name": "IndexerAgentPavel"}).stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")


Output from node 'agent':
---
{'messages': [HumanMessage(content='Build javascript code that returns all function calls with their arguments, block number, date and receiptId'), AIMessage(content="Here is the JavaScript code that you can use to extract function calls from the block:\n\n```javascript\nconst functionCalls = block.actions().flatMap((a) => \n    a.operations.filter((op) => op instanceof FunctionCall)\n    .map((op) => ({\n        blockNumber: block.blockHeight,\n        date: new Date(parseInt(block.header().timestampNanosec) / 1000000),\n        receiptId: a.receiptId,\n        methodName: op.methodName,\n        args: JSON.parse(Buffer.from(op.args).toString())\n    }))\n);\nreturn functionCalls;\n```\n\nThis code first gets all the actions in the block, then filters out the operations that are function calls. For each function call, it creates a new object with the block number, date, receiptId, method name, and arguments.\n\nYou can run this code on a block by calling 

In [73]:

from tools.JavaScriptRunner import run_js_on_block
code = """
return block.actions().map(a => ({\\n  functionCall: a.action && a.action.action && a.action.action.FunctionCall,\\n  blockHeight: block.blockHeight,\\n  date: block.header().timestampNanosec,\\n  receiptId: a.receiptId\\n}))              
"""
print(run_js_on_block(119688211, code.replace("\\n", "")))

[{'blockHeight': 119688211, 'date': '1716617107823227413', 'receiptId': '6cGAYTm4DwTZfeKFfQhMj3N15A7Fh4rPNQSpasx6p3uF'}, {'blockHeight': 119688211, 'date': '1716617107823227413', 'receiptId': 'Jfw3ubYSSV7RcwtNWNA9K1TgzGXe6fXhPk8M5DW3jPK'}, {'blockHeight': 119688211, 'date': '1716617107823227413', 'receiptId': '4P9w28PPeUZ5CVHhiyQU4vaTZmbmBHhnCXx81L6afVw4'}, {'blockHeight': 119688211, 'date': '1716617107823227413', 'receiptId': 'DFP6G4vvU6FWnW3Y6Z68PvdMyHGNxoFYpR1LRzFQsUmS'}, {'blockHeight': 119688211, 'date': '1716617107823227413', 'receiptId': '7EALz1njNXsP1HhHAU7VvpjPJV2XtyssfSfNXmGH85Ep'}, {'blockHeight': 119688211, 'date': '1716617107823227413', 'receiptId': 'B4gDLPZKCY1N3ao9NXcfie47UiQDMkFXq6ft6GK5p9J3'}, {'blockHeight': 119688211, 'date': '1716617107823227413', 'receiptId': 'GHDFcjRU6QimH1ETgXfB6McNF4eMbYymt8iUSn8hKTmy'}, {'blockHeight': 119688211, 'date': '1716617107823227413', 'receiptId': '6QMroNDKj4xe9BLNq2b7hc1EgXbXuYZku6V2nkD7iDjz'}, {'blockHeight': 119688211, 'date': '1716